In [1]:
data = r"C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\ALL FILE CSV\all_file.csv"
stopword_id = r'C:\Users\muham\OneDrive - Universitas Airlangga\Semester 6\Sistem Temu Kembali Informasi\Tugas dan Latihan\Final tugas akhir\IR\stopwords-id.txt'

In [2]:
import pandas as pd
import numpy as np
import string
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from joblib import Parallel, delayed
from joblib import Memory
import pickle

# Caching setup
location = './cacheall'
memory = Memory(location, verbose=0)

# Inisialisasi stemmer bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membaca daftar stop words bahasa Indonesia
with open(stopword_id, 'r') as f:
    stop_words_id = f.read().splitlines()

def preprocess_text(text):
    # Pastikan text adalah string
    if not isinstance(text, str):
        text = str(text)
    
    # Menghilangkan karakter berulang
    text = re.sub(r'(.)\\1+', r'\\1', text)
    
    # Menghilangkan angka
    text = ''.join([i for i in text if not i.isdigit()])
    
    # Menghilangkan tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Mengubah teks menjadi huruf kecil
    text = text.lower()
    
    # Melakukan stemming pada teks
    text = stemmer.stem(text)
    
    return text

@memory.cache
def preprocess_parallel(text_series):
    return Parallel(n_jobs=-1)(delayed(stemmer.stem)(text) for text in text_series)
# Membaca file CSV
df = pd.read_csv(data)

# Inisialisasi TfidfVectorizer dengan stop words bahasa Indonesia
tfidf = TfidfVectorizer(stop_words=stop_words_id, max_df=0.85, min_df=2, ngram_range=(1, 2))

# Melakukan fit dan transformasi pada kolom Teks
tfidf_matrix = tfidf.fit_transform(df['Teks'])

def search_documents(query, top_n=10):
    # Preprocessing query
    query = preprocess_text(query)
    
    # Transformasi query menjadi vektor tf-idf
    query_vec = tfidf.transform([query])
    
    # Menghitung cosine similarity antara query dan semua dokumen
    cosine_similarities = linear_kernel(query_vec, tfidf_matrix).flatten()
    
    # Mendapatkan indeks dokumen dengan similarity tertinggi
    related_docs_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Mendapatkan judul, teks, dan nilai similarity dari dokumen yang relevan
    results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i], i) for i in related_docs_indices]
    
    return results
def get_feedback(results):
    feedback = []
    for idx, (title, text, similarity, doc_index) in enumerate(results):
        print(f"Dokumen {idx + 1}:")
        print(f"Judul: {title}")
        print(f"Teks: {text[:200]}...")  # Menampilkan hanya 200 karakter pertama
        print(f"Similarity: {similarity:.4f}")
        relevansi = int(input("Masukkan nilai relevansi (1-5): "))
        print(relevansi)
        feedback.append((doc_index, relevansi))
    return feedback

def optimize_with_feedback(feedback, tfidf_matrix):
    relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
    non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]
    
    if not relevant_docs:
        print("Tidak ada dokumen yang dianggap relevan. Pencarian ulang tidak dapat dilakukan.")
        return None
    
    relevant_matrix = tfidf_matrix[relevant_docs]
    non_relevant_matrix = tfidf_matrix[non_relevant_docs] if non_relevant_docs else np.zeros(relevant_matrix.shape)
    
    # Hitung centroid dari dokumen yang relevan dan tidak relevan
    relevant_centroid = np.asarray(relevant_matrix.mean(axis=0)).flatten()
    non_relevant_centroid = np.asarray(non_relevant_matrix.mean(axis=0)).flatten() if non_relevant_docs else np.zeros(relevant_centroid.shape)
    
    # Fungsi untuk menyesuaikan vektor kueri dengan centroid yang relevan dan tidak relevan
    def adjust_query_vec(query_vec, relevant_centroid, non_relevant_centroid, alpha=1, beta=0.75, gamma=0.15):
        return alpha * query_vec + beta * relevant_centroid - gamma * non_relevant_centroid
    
    return adjust_query_vec
def save_optimized_query_vec(optimized_query_vec, filename):
    with open(filename, 'wb') as f:
        pickle.dump(optimized_query_vec, f)

def load_optimized_query_vec(filename):
    try:
        with open(filename, 'rb') as f:
            optimized_query_vec = pickle.load(f)
        return optimized_query_vec
    except FileNotFoundError:
        return None

if __name__ == "__main__":
    query = input("Masukkan Query: ")
    print(f"Query: {query}\n")
    print()

    optimized_query_vec = load_optimized_query_vec('optimized_query.pkl')
    if optimized_query_vec is not None:
        # Gunakan vektor kueri yang dioptimalkan untuk mencari dokumen yang relevan
        cosine_similarities = linear_kernel(optimized_query_vec, tfidf_matrix).flatten()
        related_docs_indices = cosine_similarities.argsort()[-10:][::-1]
        optimized_results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
        print("\n\n -- HASIL PENELUSURAN MENGGUNAKAN VEKTOR KUERI YANG DIOPTIMALKAN -- \n\n")
        for title, text, similarity in optimized_results:
            print(f"Judul: {title}")
            print(f"Similarity: {similarity:.4f}")
            print()
    else:
        # Lakukan pencarian seperti biasa dan minta umpan balik dari pengguna
        initial_results = search_documents(query)
        feedback = get_feedback(initial_results)
        adjust_query_vec = optimize_with_feedback(feedback, tfidf_matrix)

        if adjust_query_vec:
            relevant_docs = [idx for idx, relevansi in feedback if relevansi >= 3]
            non_relevant_docs = [idx for idx, relevansi in feedback if relevansi < 3]

            # Reprocess the query with the adjusted query vector
            query_vec = tfidf.transform([preprocess_text(query)])
            adjusted_query_vec = adjust_query_vec(query_vec.toarray(), np.asarray(tfidf_matrix[relevant_docs].mean(axis=0)).flatten(), 
                                                  np.asarray(tfidf_matrix[non_relevant_docs].mean(axis=0)).flatten() if non_relevant_docs else np.zeros(query_vec.shape))
            
            # Compute cosine similarity with the adjusted query vector
            cosine_similarities = linear_kernel(adjusted_query_vec, tfidf_matrix).flatten()
            related_docs_indices = cosine_similarities.argsort()[-10:][::-1]
            
            # Display optimized results
            optimized_results = [(df.iloc[i]['Judul'], df.iloc[i]['Teks'], cosine_similarities[i]) for i in related_docs_indices]
            print("\n\n -- HASIL PENELUSURAN ULANG -- \n\n")
            for title, text, similarity in optimized_results:
                print(f"Judul: {title}")
                print(f"Similarity: {similarity:.4f}")
                print()
            
            save_optimized_query_vec(adjusted_query_vec, 'optimized_query.pkl')

Query: pencurian


Dokumen 1:
Judul: Undang-Undang Nomor 9 Tahun 2023
Teks: MenimbangSALINANPRESIDENREPUBLIK INDONESIAUNDANG-UNDANG REPUBLIK TNDONESIANOMOR 9 TAHUN 2023TENTANGPROVINSI SUMATERA SEI.,ATANDENGAN RAHMAT TUHAN YANG MAHA ESAPRESIDEN REPUBLIK INDONESIA,a. bahwa prov...
Similarity: 0.0000
1
Dokumen 2:
Judul: Undang-Undang Nomor 9 Tahun 2022
Teks:    LEMBARAN NEGARA  REPUBLIK INDONESIA  No.69, 2022  OTONOMI. PEMERINTAH . Provinsi  Kalimantan Barat . Pencabutan.  (Penjelasan dalam Tambahan Lembaran Negara Republik Indonesia Nomor 6780)  UNDANG -...
Similarity: 0.0000
5
Dokumen 3:
Judul: Undang-Undang Nomor 2 Tahun 1983
Teks:   UNDANG-UNDANG REPUBLIK INDONESIA NOMOR 2 TAHUN 1983 TENTANG ANGGARAN PENDAPATAN DAN BELANJA NEGARA TAHUN ANGGARAN 1983/1984  DENGAN RAHMAT TUHAN YANG MAHA ESA  PRESIDEN REPUBLIK INDONESIA,  Menimban...
Similarity: 0.0000
3
Dokumen 4:
Judul: Undang-Undang Nomor 2 Tahun 1984
Teks:   UNDANG-UNDANG REPUBLIK INDONESIA NOMOR 2 TAHUN 1984 TENTANG PERHITUNGAN ANGG